## SNOTEL

In [1]:
import json
import requests
import pandas as pd
import geopandas as gpd
from sedona.spark import *
from pyspark.sql.functions import expr

## Sedona Setup

In [2]:
config = SedonaContext.builder().\
    config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"). \
    getOrCreate()

sedona = SedonaContext.create(config)

24/03/28 20:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 20:22:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/03/28 20:22:41 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to qjnq6fcbf1/spark-logs/spark-07269197f47b435a90df227d70965071.inprogress. This is unsupported
24/03/28 20:23:03 WARN SedonaContext: Python files are not set. Sedona will not pre-load Python UDFs.


## SNOTEL Rest API

In [3]:
baseurl = "https://wcc.sc.egov.usda.gov/awdbRestApi/"

## Stations

parameters

In [4]:
payload = {
    'activeOnly': True,
    'returnStationElements': False
}

In [5]:
r = requests.get(baseurl + 'services/v1/stations', params=payload)

In [6]:
stations = r.json()

In [7]:
len(stations)

4363

In [8]:
df = pd.DataFrame(stations)

In [9]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [10]:
gdf

,stationTriplet,stationId,stateCode,networkCode,name,dcoCode,countyName,huc,elevation,latitude,longitude,dataTimeZone,pedonCode,shefId,beginDate,endDate,geometry
0,2057:AL:SCAN,2057,AL,SCAN,AAMU-JTG,GC,Madison,060300020403,860.0,34.78333,-86.55000,-6.0,27979,AAMA1,2002-02-23 00:00:00.0,2100-01-01 00:00:00.0,POINT (-86.55000 34.78333)
1,ABY:CA:SNOW,ABY,CA,SNOW,Abbey,UN,Plumas,180201220103,5650.0,39.95500,-120.53800,NaN,NaN,NaN,1963-02-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-120.53800 39.95500)
2,0010:ID:COOP,0010,ID,COOP,Aberdeen Experimnt Stn,ID,Bingham,170402060704,4410.0,42.95000,-112.83333,NaN,NaN,ABDI1,1914-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-112.83333 42.95000)
3,1F01A:BC:SNOW,1F01A,BC,SNOW,Aberdeen Lake,OR,UNKNOWN,NaN,4298.0,50.14733,-119.05340,NaN,NaN,ABLQ2,1939-04-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-119.05340 50.14733)
4,0041:NM:COOP,0041,NM,COOP,Abiquiu Dam,UN,Rio Arriba,130201021005,6380.0,36.23333,-106.43333,NaN,NaN,ABIN5,1957-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-106.43333 36.23333)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4358,YBP:CA:SNOW,YBP,CA,SNOW,Yuba Pass,UN,Sierra,180201250202,6700.0,39.61700,-120.49200,NaN,NaN,NaN,1937-03-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-120.49200 39.61700)
4359,15356000:AK:USGS,15356000,AK,USGS,Yukon R at Eagle,AK,Southeast Fairbanks,190705051301,850.0,64.78917,-141.20009,NaN,NaN,NaN,1950-01-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-141.20009 64.78917)
4360,15453500:AK:USGS,15453500,AK,USGS,Yukon R nr Stevens Village,AK,Yukon-Koyukuk,190804040504,240.0,65.87510,-149.72035,NaN,NaN,NaN,1976-10-01 00:00:00.0,2100-01-01 00:00:00.0,POINT (-149.72035 65.87510)
4361,1033:CO:SNTL,1033,CO,SNTL,Zirkel,CO,Jackson,101800010202,9338.0,40.79492,-106.59544,-8.0,NaN,ZIRC2,2002-08-14 00:00:00.0,2100-01-01 00:00:00.0,POINT (-106.59544 40.79492)


In [11]:
station_df = sedona.createDataFrame(gdf)
station_df.createOrReplaceTempView('stations')

In [12]:
station_df.printSchema()

root
 |-- stationTriplet: string (nullable = true)
 |-- stationId: string (nullable = true)
 |-- stateCode: string (nullable = true)
 |-- networkCode: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dcoCode: string (nullable = true)
 |-- countyName: string (nullable = true)
 |-- huc: string (nullable = true)
 |-- elevation: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- dataTimeZone: double (nullable = true)
 |-- pedonCode: string (nullable = true)
 |-- shefId: string (nullable = true)
 |-- beginDate: string (nullable = true)
 |-- endDate: string (nullable = true)
 |-- geometry: geometry (nullable = true)



In [13]:
station_df.show()

+---------------+---------+---------+-----------+--------------------+-------+------------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+
| stationTriplet|stationId|stateCode|networkCode|                name|dcoCode|  countyName|         huc|elevation|latitude| longitude|dataTimeZone|pedonCode|shefId|           beginDate|             endDate|            geometry|
+---------------+---------+---------+-----------+--------------------+-------+------------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+
|   2057:AL:SCAN|     2057|       AL|       SCAN|            AAMU-JTG|     GC|     Madison|060300020403|    860.0|34.78333|    -86.55|        -6.0|    27979| AAMA1|2002-02-23 00:00:...|2100-01-01 00:00:...|POINT (-86.55 34....|
|    ABY:CA:SNOW|      ABY|       CA|       SNOW|               Abbey|     UN|      Plum

In [14]:
sedona.sql("SELECT COUNT(*) FROM stations WHERE networkCode = 'SNTL'").show()

+--------+
|count(1)|
+--------+
|     899|
+--------+



In [15]:
states = sedona.sql("SELECT distinct(stateCode) FROM stations WHERE networkCode = 'SNTL'").toPandas().values.tolist()
states = [x[0] for x in states]

In [16]:
states

['AZ', 'OR', 'WY', 'NV', 'ID', 'CA', 'MT', 'WA', 'CO', 'AK', 'UT', 'NM', 'SD']

## Data

parameters

* `beginDate` - yyyy-MM-dd HH:mm
* `centralTendencyType` - MEDIAN
* `duration` - DAILY
* `elements` - PREC, WTEQ, SMS:*, PREC::2
* `endDate` - yyyy-MM-dd HH:mm
* `stationTriplets` - *:*:*

In [17]:
station_data = []
for state in states:
    payload = {
        'beginDate': '2024-03-26',
        'endDate': '2024-03-26',
        'centralTendencyType': 'MEDIAN',
        'duration': 'DAILY',
        'elements': 'WTEQ',
        'stationTriplets': f"*:{state}:SNTL"
    }
    r = requests.get(baseurl + 'services/v1/data', params=payload)
    station_data = station_data + r.json()

In [18]:
len(station_data)

854

In [19]:
df = pd.DataFrame(station_data)

In [20]:
df

,stationTriplet,data
0,308:AZ:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
1,1140:AZ:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
2,310:AZ:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
3,1212:AZ:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
4,902:AZ:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
...,...,...
849,1083:NM:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
850,1017:NM:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
851,854:NM:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."
852,354:SD:SNTL,"[{'stationElement': {'elementCode': 'WTEQ', 'o..."


In [21]:
data_list = [(station['stationTriplet'], station['data'][0]['values'][0]['value'], station['data'][0]['values'][0].get('median'), station['data'][0]['values'][0]['date']) for station in station_data]

In [22]:
df = pd.DataFrame(data_list, columns=['station', 'value', 'median', 'date'])

In [23]:
df

,station,value,median,date
0,308:AZ:SNTL,5.8,0.0,2024-03-26
1,1140:AZ:SNTL,19.4,11.7,2024-03-26
2,310:AZ:SNTL,6.6,2.2,2024-03-26
3,1212:AZ:SNTL,1.0,NaN,2024-03-26
4,902:AZ:SNTL,0.4,0.0,2024-03-26
...,...,...,...,...
849,1083:NM:SNTL,1.3,0.0,2024-03-26
850,1017:NM:SNTL,13.8,5.7,2024-03-26
851,854:NM:SNTL,14.1,12.0,2024-03-26
852,354:SD:SNTL,0.2,6.8,2024-03-26


In [24]:
data_df = sedona.createDataFrame(df)

In [25]:
data_df.show()

+------------+-----+------+----------+
|     station|value|median|      date|
+------------+-----+------+----------+
| 308:AZ:SNTL|  5.8|   0.0|2024-03-26|
|1140:AZ:SNTL| 19.4|  11.7|2024-03-26|
| 310:AZ:SNTL|  6.6|   2.2|2024-03-26|
|1212:AZ:SNTL|  1.0|   NaN|2024-03-26|
| 902:AZ:SNTL|  0.4|   0.0|2024-03-26|
|1143:AZ:SNTL| 12.3|   3.7|2024-03-26|
|1139:AZ:SNTL|  0.7|   0.0|2024-03-26|
| 416:AZ:SNTL|  1.4|   0.0|2024-03-26|
|1121:AZ:SNTL|  0.9|   0.0|2024-03-26|
| 488:AZ:SNTL|  6.1|   0.0|2024-03-26|
| 511:AZ:SNTL| 11.7|   5.0|2024-03-26|
| 969:AZ:SNTL|  8.6|   2.1|2024-03-26|
|1271:AZ:SNTL| 17.2|   NaN|2024-03-26|
| 519:AZ:SNTL|  2.2|   0.0|2024-03-26|
| 617:AZ:SNTL|  7.9|   4.2|2024-03-26|
| 640:AZ:SNTL|  4.4|   0.4|2024-03-26|
|1125:AZ:SNTL| 10.7|   7.2|2024-03-26|
|1127:AZ:SNTL|  0.8|   0.0|2024-03-26|
| 705:AZ:SNTL|  7.4|   3.9|2024-03-26|
| 927:AZ:SNTL| 22.2|  14.4|2024-03-26|
+------------+-----+------+----------+
only showing top 20 rows



In [26]:
data_df.createOrReplaceTempView('snotel_data')

In [27]:
## Join Daily Observations 


snotel_data_df = sedona.sql("""
SELECT *, (value / median) AS pct_median
FROM stations
JOIN snotel_data 
WHERE snotel_data.station = stations.stationTriplet
""")

In [28]:
snotel_data_df.createOrReplaceTempView("snotel_data")

In [29]:
snotel_data_df.count()

854

In [30]:
snotel_data_df.show()

+--------------+---------+---------+-----------+--------------------+-------+----------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+------------+-----+------+----------+------------------+
|stationTriplet|stationId|stateCode|networkCode|                name|dcoCode|countyName|         huc|elevation|latitude| longitude|dataTimeZone|pedonCode|shefId|           beginDate|             endDate|            geometry|     station|value|median|      date|        pct_median|
+--------------+---------+---------+-----------+--------------------+-------+----------+------------+---------+--------+----------+------------+---------+------+--------------------+--------------------+--------------------+------------+-----+------+----------+------------------+
|   969:AZ:SNTL|      969|       AZ|       SNTL|          Happy Jack|     CO|  Coconino|150602020604|   7539.0|34.74594|-111.41219|        -8.0|      NaN| XH

In [31]:
SedonaKepler.create_map(snotel_data_df)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/opt/conda/lib/python3.10/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…

## Watershed - Spatial Join

In [32]:
# TODO: load geojson, spatial join, calculate 

In [33]:
WATERSHED_URI = "s3://wherobots-examples/data/examples/usgs/watershed_boundaries_huc6.geojson"

In [34]:
watershed_df = sedona.read.format("geojson").option("multiLine", "true").load(WATERSHED_URI).selectExpr("explode(features) AS features").select("features.*").withColumn("huc6", expr("properties['huc6']")).withColumn("name", expr("properties['name']")).withColumn("states", expr("properties['states']")).drop("properties").drop("type")

In [35]:
watershed_df.show()

+--------------------+------+--------------------+--------+
|            geometry|  huc6|                name|  states|
+--------------------+------+--------------------+--------+
|MULTIPOLYGON (((-...|050902|Middle Ohio-Littl...|IN,KY,OH|
|MULTIPOLYGON (((-...|030201|             Pamlico|      NC|
|MULTIPOLYGON (((-...|031501|    Coosa-Tallapoosa|AL,GA,TN|
|MULTIPOLYGON (((-...|050800|         Great Miami|   IN,OH|
|MULTIPOLYGON (((-...|051201|              Wabash|IL,IN,OH|
|MULTIPOLYGON (((-...|031403|            Escambia|   AL,FL|
|MULTIPOLYGON (((-...|051202|        Patoka-White|      IN|
|MULTIPOLYGON (((-...|111402|          Red-Saline|AR,LA,TX|
|MULTIPOLYGON (((-...|080702|       Lake Maurepas|   LA,MS|
|MULTIPOLYGON (((-...|080902|  Lake Pontchartrain|   LA,MS|
|MULTIPOLYGON (((-...|120402|Galveston Bay-Sab...|   LA,TX|
|MULTIPOLYGON (((-...|121004|Central Texas Coa...|      TX|
|MULTIPOLYGON (((-...|010600|                Saco|MA,ME,NH|
|MULTIPOLYGON (((-...|140200|           

In [36]:
watershed_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- huc6: string (nullable = true)
 |-- name: string (nullable = true)
 |-- states: string (nullable = true)



In [37]:
watershed_df.createOrReplaceTempView("watersheds")

### Spatial Join - SNOTEL data to Watersheds

In [41]:
snotel_sheds_df = sedona.sql("""
SELECT any_value(watersheds.name) AS name, any_value(states) AS states, any_value(watersheds.geometry) AS geometry, avg(pct_median) AS pct_median
FROM watersheds, snotel_data
WHERE ST_Contains(watersheds.geometry, snotel_data.geometry) AND snotel_data.pct_median IS NOT null AND NOT isnan(pct_median)
GROUP BY huc6
""")

In [42]:
snotel_sheds_df.show()

+--------------------+-----------+--------------------+------------------+
|                name|     states|            geometry|        pct_median|
+--------------------+-----------+--------------------+------------------+
| Missouri Headwaters|   ID,MT,WY|MULTIPOLYGON (((-...|0.8123762185493943|
|         Musselshell|         MT|MULTIPOLYGON (((-...|0.7634408602150536|
|        South Platte|   CO,NE,WY|MULTIPOLYGON (((-...|1.2160896318379226|
|      Upper Arkansas|   CO,KS,NM|MULTIPOLYGON (((-...|1.4231765640047371|
|      Upper San Juan|   AZ,CO,NM|MULTIPOLYGON (((-...|10.034949087541726|
|     Little Colorado|      AZ,NM|MULTIPOLYGON (((-...|3.7850427350427345|
|            Knik Arm|         AK|MULTIPOLYGON (((-...|1.2429124016080537|
|        North Platte|   CO,NE,WY|MULTIPOLYGON (((-...|0.9679540180633677|
|               Verde|         AZ|MULTIPOLYGON (((-...|2.8766788766788762|
|Central Nevada De...|      CA,NV|MULTIPOLYGON (((-...|1.7738231188935414|
|       Koyukuk River|   

In [43]:
SedonaKepler.create_map(snotel_sheds_df)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,…